<a href="https://colab.research.google.com/github/AleksandrRevuka/Algo_data_science/blob/main/7_Practical_skills_1_2_filling_in_data_gaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filling in data gaps:

In [1]:
import pandas as pd
import numpy as np

from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

In [3]:
energy = energy[energy['building_id'] == 0]

In [4]:
energy = pd.merge(left=energy, right=buildings, how="left", left_on="building_id", right_on="building_id")

energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])

energy = pd.merge(left=energy, right=weather, how="left", left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "floor_count"], axis=1)

del buildings
del weather

energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   timestamp           8784 non-null   object 
 1   building_id         8784 non-null   int64  
 2   meter_reading       8784 non-null   float64
 3   primary_use         8784 non-null   object 
 4   square_feet         8784 non-null   int64  
 5   year_built          8784 non-null   float64
 6   air_temperature     8781 non-null   float64
 7   cloud_coverage      4954 non-null   float64
 8   dew_temperature     8781 non-null   float64
 9   precip_depth_1_hr   8783 non-null   float64
 10  sea_level_pressure  8699 non-null   float64
 11  wind_direction      8534 non-null   float64
 12  wind_speed          8784 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 892.2+ KB


In [5]:
def reduce_mem_usage(df):
  start_mem = df.memory_usage().sum() / 1024 ** 2
  for col in df.columns:
    col_type = df[col].dtypes
    if str(col_type)[:5] == "float":
      c_min = df[col].min()
      c_max = df[col].max()
      if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
        df[col] = df[col].astype(np.float16)
      elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
        df[col] = df[col].astype(np.float32)
      else:
        df[col] = df[col].astype(np.float64)
    elif str(col_type)[:3] == "int":
        c_min = df[col].min()
        c_max = df[col].max()
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
            df[col] = df[col].astype(np.int64)
    elif col == "timestamp":
      df[col] = pd.to_datetime(df[col])
    elif str(col_type)[:8] != "dataetime":
      df[col] = df[col].astype("category")
  end_mem = df.memory_usage().sum() / 1024 ** 2
  print(f"Споживання памяті меньше на {round(start_mem - end_mem, 2)}Mb (мінус {round(100 * (start_mem - end_mem) / start_mem, 1)}%)")
  return df

In [6]:
energy = reduce_mem_usage(energy)

Споживання памяті меньше на 0.62Mb (мінус 71.1%)


In [7]:
energy['precip_depth_1_hr'] = energy['precip_depth_1_hr'].apply(lambda x: x if x > 0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature", "cloud_coverage", "wind_speed", "precip_depth_1_hr", "sea_level_pressure"]

for col in interpolate_columns:
  energy[col] = energy[col].interpolate(limit_direction="both", kind="cubic")

In [8]:
pd.set_option("use_inf_as_na", True)
for col in interpolate_columns:
  print(f"{col} Inf+NaN: {energy[col].isnull().sum()}")

air_temperature Inf+NaN: 0
dew_temperature Inf+NaN: 0
cloud_coverage Inf+NaN: 0
wind_speed Inf+NaN: 0
precip_depth_1_hr Inf+NaN: 0
sea_level_pressure Inf+NaN: 0


In [10]:
energy_train, energy_test = train_test_split(energy[energy['meter_reading'] > 0], test_size=0.2)
energy_train

,timestamp,building_id,meter_reading,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
4154,2016-06-22 02:00:00,0,222.500,Education,7432,2008.0,25.000000,4.000000,18.906250,0.0,1020.0,120.0,3.099609
4923,2016-07-24 03:00:00,0,245.750,Education,7432,2008.0,25.000000,4.500000,22.796875,0.0,1017.5,0.0,0.000000
6103,2016-09-11 07:00:00,0,247.125,Education,7432,2008.0,25.593750,4.000000,24.406250,0.0,1018.0,0.0,0.000000
4095,2016-06-19 15:00:00,0,296.250,Education,7432,2008.0,28.906250,4.000000,21.703125,0.0,1020.5,50.0,7.199219
5256,2016-08-07 00:00:00,0,255.250,Education,7432,2008.0,26.703125,6.000000,21.703125,0.0,1014.5,300.0,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6816,2016-10-11 00:00:00,0,223.875,Education,7432,2008.0,22.796875,8.000000,15.000000,0.0,1021.0,20.0,5.101562
6818,2016-10-11 02:00:00,0,224.500,Education,7432,2008.0,21.703125,5.332031,16.093750,0.0,1021.5,10.0,5.101562
6187,2016-09-14 19:00:00,0,231.375,Education,7432,2008.0,30.000000,6.332031,23.296875,0.0,1015.5,170.0,3.599609
5468,2016-08-15 20:00:00,0,301.000,Education,7432,2008.0,30.000000,6.000000,23.906250,0.0,1019.0,180.0,2.099609


In [11]:
regression_columns = ["meter_reading", "air_temperature", "dew_temperature", "cloud_coverage", "wind_speed", "precip_depth_1_hr", "sea_level_pressure"]

energy_train_lr = pd.DataFrame(energy_train, columns=regression_columns)

y = energy_train_lr["meter_reading"]
x = energy_train_lr.drop(labels=["meter_reading"], axis=1)
model = LinearRegression().fit(x, y)
print(model.coef_, model.intercept_)

[ 2.55636775  3.58554422 -2.30214615 -1.85989461  0.11045287 -0.87774046] 1013.0350566191762


In [12]:
def calculate_model(x):
  lr = np.sum([x[col] * model.coef_[i] for i, col in enumerate(regression_columns[1:])])
  lr += model.intercept_
  x["meter_reading_lr_q"] = (np.log(1 + x.meter_reading) - np.log(1 + lr)) ** 2
  return x

energy_test = energy_test.apply(calculate_model, axis=1, result_type="expand")
energy_train_lr_rmsle = np.sqrt(energy_test['meter_reading_lr_q'].sum() / len(energy_test))
print (f"Якість лінійної регресії: {energy_train_lr_rmsle} {round(energy_train_lr_rmsle, 1)}")

Якість лінійної регресії: 0.22586444236388548 0.2
